In [1]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [2]:
import tensorflow_datasets as tfds
dataset = tfds.load("fashion_mnist")
dataset

2021-04-13 17:12:16,683 : INFO : Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: fashion_mnist/3.0.1
2021-04-13 17:12:18,150 : INFO : Load dataset info from /var/folders/g9/6qklj4h53bv0c1rjnffg7bmw0000gp/T/tmptx4kpgi0tfds
2021-04-13 17:12:18,153 : INFO : Field info.citation from disk and from code do not match. Keeping the one from code.
2021-04-13 17:12:18,153 : INFO : Field info.splits from disk and from code do not match. Keeping the one from code.
2021-04-13 17:12:18,154 : INFO : Field info.module_name from disk and from code do not match. Keeping the one from code.
2021-04-13 17:12:18,154 : INFO : Generating dataset fashion_mnist (/Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1)


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html